In [1]:
import os
import numpy as np
import pandas as pd
import time
import httplib2
import urllib
import requests
import re
from bs4 import BeautifulSoup

# Imports

# Config

In [239]:
class Config():
    def __init__(self, data_path='/Users/guillaumecorda/Desktop/UvA/Information Retrieval/Project/data/', url=None):
        self.data_path = data_path
        self.url = url

In [240]:
#addd user-agent in url user-agent=bot
cfg_ams = Config(url='https://www.huurda.com/Housing_offer.html?searchPhotos=1&searchKeywords=amsterdam&searchRentStart=0&searchSurface=0&searchRentEnd=0&searchAvailable=&searchFurnished=&searchBedrooms=&searchFormSubmit=Search+directly+in+6.510+rental+houses')
cfg_rot = Config(url='https://www.huurda.com/Housing_offer.html?searchPhotos=1&searchKeywords=rotterdam&searchRentStart=0&searchSurface=0&searchRentEnd=0&searchAvailable=&searchFurnished=&searchBedrooms=&searchFormSubmit=Search+directly+in+6.518+rental+houses')
cfg_haag = Config(url='https://www.huurda.com/Housing_offer.html?searchPhotos=1&searchKeywords=den+haag&searchRentStart=0&searchSurface=0&searchRentEnd=0&searchAvailable=&searchFurnished=&searchBedrooms=&searchFormSubmit=Search+directly+in+6.518+rental+houses')
cfg_gro = Config(url='https://www.huurda.com/Housing_offer.html?searchPhotos=1&searchKeywords=groningen&searchRentStart=0&searchSurface=0&searchRentEnd=0&searchAvailable=&searchFurnished=&searchBedrooms=&searchFormSubmit=Search+directly+in+6.517+rental+houses')
cfg_utr = Config(url='https://www.huurda.com/Housing_offer.html?searchPhotos=1&searchKeywords=utrecht&searchRentStart=0&searchSurface=0&searchRentEnd=0&searchAvailable=&searchFurnished=&searchBedrooms=&searchFormSubmit=Search+directly+in+6.517+rental+houses')
cfg_ein = Config(url='https://www.huurda.com/Housing_offer.html?searchPhotos=1&searchKeywords=eindhoven&searchRentStart=0&searchSurface=0&searchRentEnd=0&searchAvailable=&searchFurnished=&searchBedrooms=&searchFormSubmit=Search+directly+in+6.517+rental+houses')

# Helper functions

## Get content

In [4]:
def get_html(url):
    if 'http' not in url:
        raise ValueError('Please add protocol to url.')
    html_page = urllib.request.urlopen(url)
    soup = BeautifulSoup(html_page, features='html.parser')
    return soup

## Remove html tags

In [5]:
def remove_html_tags(text):
    p = re.compile(r'<.*?>')
    return p.sub('', text)

## Extract outgoing links

In [6]:
def extract_outgoing_links(url):
    soup = get_html(url)
    content = soup.find_all('div', class_ = 'newsBox')
    links = list()
    for flat in content:
        tag_news = str(flat.find('a', class_='news'))
        url = re.findall('"([^"]*)"', tag_news)[1]
        links.append(url)
    return links

## Url editor

In [7]:
def edit_url(url, page):
    if 'page' not in url :
        url = url+ '&page='+str(page)
    else:
        url = url.replace(url[-1], str(page))
    return url

## Details Scraper

In [8]:
def get_rent_details(url):
    soup = get_html(url)
    details_rent = soup.find_all('div', class_ = 'detailBox')

    infos = remove_html_tags(str(details_rent[0]))
    infos = infos.replace('\n', ' ')
    infos = infos.replace('\t', ' ')
    infos = infos.replace('\n2', ' ')
    infos = infos.split('   ')
    infos = [el for el in infos if el != '']
    
    infos = infos[1:8]
    
    #uncomment lines for new features
    infos[0] = infos[0].replace('Rent per month: € ','')
    infos[0] = infos[0].replace(',-', '')
    infos[1] = infos[1].replace('Available from: ', '')
    #infos[2] = infos[2].replace('Offered since: ', '')
    #infos[3] = infos[3].replace('Offered by: ', '')
    infos[4] = infos[4].replace('Surface (m2): ', '')
    infos[4] = infos[4].replace(' m²', '')
    infos[5] = infos[5].replace('Interior:  ', '')
    infos[6] = infos[6].replace('Number of bedrooms ', '')

    #remove line for new features
    del infos[2:3]
    del infos[2]
    
    return infos

## Location Scraper

In [158]:
def get_location(url, city):
    
    soup = get_html(url)
    content = soup.find_all('div', class_ = 'brownBox')

    infos = remove_html_tags(str(content[0]))
    infos = infos.replace('\n', ' ')
    infos = infos.replace('\t', ' ')
    infos = infos.replace('\r', ' ')
    infos = infos.replace('\n2', ' ')
    infos = infos.lower()
    
    if city == 'Amsterdam':
        indice_start = infos.rfind('in amsterdam')
        infos = infos[indice_start+len('in amsterdam'):]
        infos = infos.replace('amsterdam', 'Amsterdam')
    elif city == 'Rotterdam':
        indice_start = infos.rfind('in rotterdam')
        infos = infos[indice_start+len('in rotterdam'):]
        infos = infos.replace('rotterdam', 'Rotterdam')
    elif city == 'Den Haag':
        indice_start = infos.rfind('in den haag')
        infos = infos[indice_start+len('in den haag'):]
        infos = infos.replace('den haag', 'Den Haag')
    elif city == 'Utrecht':
        indice_start = infos.rfind('in utrecht')
        infos = infos[indice_start+len('in utrecht'):]
        infos = infos.replace('utrecht', 'Utrecht')
    elif city == 'Groningen':
        indice_start = infos.rfind('in groningen')
        infos = infos[indice_start+len('in groningen'):]
        infos = infos.replace('groningen', 'Groningen')
    elif city == 'Eindhoven':
        indice_start = infos.rfind('eindhoven')
        infos = infos[indice_start+len('eindhoven'):]
        infos = infos.replace('eindhoven', 'Eindhoven')
    
    infos = infos.strip()
    indice_end = infos.find('€')
    infos = infos[:indice_end]
    location = " ".join(infos.split())
    
    
    return location

## Description Scraper

In [191]:
def get_description(url, city):
    soup = get_html(url)
    details_rent = soup.find_all('div', class_ = 'contentbox')
    
    infos = remove_html_tags(str(details_rent[0]))
    infos = infos.replace('\n', ' ')
    infos = infos.replace('\t', ' ')
    infos = infos.replace('\n2', ' ')
    infos = infos.split('   ')
    
    if len(infos)==1:
        infos = infos[0].strip()
    
    elif len(infos) > 1:
        infos = ''.join(str(elem) for elem in infos)
        infos = infos.strip()
    
    infos = infos.lower()
    
    if '123wonen the rental agency offers:' in infos:
        infos = infos.replace('123wonen the rental agency offers:', '')
    elif '123wonen the rental agent offers:' in infos:
        infos = infos.replace('123wonen the rental agent offers:', '')
    
    if ',-' in infos:
        indice_start = infos.find(',-')
        infos = infos[indice_start+2:]
    
    elif ', -' in infos:
        indice_start = infos.find(', -')
        infos = infos[indice_start+3:]
    
    if 'all in' in infos:
        indice_start = infos.find('all in')
        infos = infos[indice_start+6:]
    
    if 'description' in infos:
        indice_start = infos.find('description')
        infos = infos[indice_start+len('description'):]
    
    infos = infos.strip()
    
    return infos

In [192]:
get_description('https://www.huurda.com/properties_for_rent_in_Utrecht/One-family_living/Janskerkhof--11757/', 'Utrecht')

':  fully furnished\xa0 2 room apartment. this apartment has a big living room and dining area an open kitchen. separate toilet at the entrance. spacious bedroom with an adjoining bathroom. the apartment has a spacious garden. this is a beautiful and apartment suitable for a short or long term stay.classification:  layout: entrance, toilet, living room approx 35m2 with sofa, coffee table, tv and tv cabinet, chairs, stereo, 2 standing lamps. kitchen with a refrigerator, stove with 4 burners, microwave, toaster and coffee/water cooker. bedroom approx 16m2 with double bed, bedside table and lamp, bathroom with shower, bath, 2nd toilet, sink and washing machine.location:  the apartment is located on the jans kerkhof heart of the city center. it has a beautiful view of the cathedral and in the surrounding area all supplies nearby.  specialties:  - total area approx 65m2  - no pets allowed  - rental period: min. 3 months \xa0 - deposit: equal to 1 month rent  - mediation costs is one months 

In [173]:
url = 'https://www.huurda.com/properties_for_rent_in_Utrecht/One-family_living/Janskerkhof--11757/'

In [187]:
soup = get_html(url)
details_rent = soup.find_all('div', class_ = 'contentbox')

infos = remove_html_tags(str(details_rent[0]))
infos = infos.replace('\n', ' ')
infos = infos.replace('\t', ' ')
infos = infos.replace('\n2', ' ')
infos = infos.split('   ')

infos = ''.join(str(elem) for elem in infos)
infos = infos.strip()

if 'all in' in infos:
    indice_start = infos.find('all in')
    infos = infos[indice_start+6:]

In [188]:
infos

'.  Description:  Fully furnished\xa0 2 room apartment. This apartment has a big living room and dining area an open kitchen. Separate toilet at the entrance. Spacious bedroom with an adjoining bathroom. The apartment has a spacious garden. This is a beautiful and apartment suitable for a short or long term stay.Classification:  Layout: entrance, toilet, living room approx 35m2 with sofa, coffee table, tv and TV cabinet, chairs, stereo, 2 standing lamps. Kitchen with a refrigerator, stove with 4 burners, microwave, toaster and coffee/water cooker. Bedroom approx 16m2 with double bed, bedside table and lamp, bathroom with shower, bath, 2nd toilet, sink and washing machine.Location:  The apartment is located on the Jans Kerkhof heart of the city center. It has a beautiful view of the cathedral and in the surrounding area all supplies nearby.  Specialties:  - Total area approx 65m2  - No pets allowed  - Rental period: min. 3 months \xa0 - Deposit: equal to 1 month rent  - Mediation costs 

# Scraper

## Scrap one link

In [ ]:
links_ams = extract_outgoing_links(cfg_ams.url)

In [ ]:
data = get_rent_details(links_ams[10])

In [ ]:
data.append(get_location(links_ams[0], city='Amsterdam'))

In [ ]:
data

## Automation for one page

In [144]:
def mini_scraper(url, city):
    
    links = extract_outgoing_links(url)
    df = pd.DataFrame(columns=['Rent', 'Available from', 'Surface', 'Interior', 'Number of bedrooms',  'Location', 'Description', 'url'])
    
    for i, link in enumerate(links):
        try:
            data = get_rent_details(link)
            data.append(get_location(link, city))
            data.append(get_description(link, city))
            data.append(link)
            df.loc[i] = data
            print('Scraping url {}/{}'.format(i+1, len(links)), end='\r')
        except:
            print(link)
    return df

In [ ]:
df = mini_scraper(cfg_rot.url, city='Rotterdam')
df.head()

## Main Scraper

In [94]:
def main_scraper(url, city, nb_page):
    print('Scraping page: 1/{}'.format(nb_page))
    df = mini_scraper(url, city)
    df_list = [df]
    for i in range(1, nb_page):
        print('Scraping page: {}/{}'.format(i+1, nb_page))
        url = edit_url(url, i)
        df_list.append(mini_scraper(url, city))
        #time.sleep(1)
    df = pd.concat(df_list)
    return df

### Amsterdam

In [198]:
df_ams = main_scraper(cfg_ams.url, city='Amsterdam', nb_page=8)

Scraping page: 1/8
https://www.huurda.com/properties_for_rent_in_Amsterdam/Apartment/CuraÃ§aostraat--404633/
https://www.huurda.com/properties_for_rent_in_Amsterdam/Apartment/BethaniÃ«ndwarsstraat--403731/
Scraping page: 2/8
https://www.huurda.com/properties_for_rent_in_Amsterdam/Apartment/CuraÃ§aostraat--404633/
https://www.huurda.com/properties_for_rent_in_Amsterdam/Apartment/BethaniÃ«ndwarsstraat--403731/
Scraping page: 3/8
Scraping page: 4/8
Scraping page: 5/8
Scraping page: 6/8
Scraping page: 7/8
Scraping page: 8/8
https://www.huurda.com/properties_for_rent_in_Amsterdam/Apartment/Sint_LuciÃ«nsteeg--278166/


In [199]:
df_ams.shape

(155, 8)

In [202]:
df_ams.head()

,Rent,Available from,Surface,Interior,Number of bedrooms,Location,Description,url
0,1.950,15-03-2019,86,furnished,2,knsm-laan 1019ld Amsterdam,modern and comfortable fully furnished apartme...,https://www.huurda.com/properties_for_rent_in_...
1,3.500,right away,130,furnished,4,cornelis anthoniszstraat 1071 vp Amsterdam,superb and completely renovated house in a pop...,https://www.huurda.com/properties_for_rent_in_...
2,1.995,right away,60,furnished,1,herengracht 1015 bt Amsterdam,fully furnished apartment in the middle of the...,https://www.huurda.com/properties_for_rent_in_...
4,1.675,11-03-2019,60,furnished,1,schinkelkade 1075 vh Amsterdam,on a top location in the old south near the vo...,https://www.huurda.com/properties_for_rent_in_...
5,1.450,right away,86,furnished,2,meeuwenlaan 1021 jc Amsterdam,the property is centrally located; pont ijplei...,https://www.huurda.com/properties_for_rent_in_...


In [203]:
df_ams.to_csv(cfg_ams.data_path + 'ams_data.csv', encoding='utf-8', index=False)

### Rotterdam

In [204]:
df_rot = main_scraper(cfg_rot.url, city='Rotterdam', nb_page=13)

Scraping page: 1/13
Scraping page: 2/13
Scraping page: 3/13
Scraping page: 4/13
Scraping page: 5/13
Scraping page: 6/13
Scraping page: 7/13
Scraping page: 8/13
Scraping page: 9/13
Scraping page: 10/13
Scraping page: 11/13
Scraping page: 12/13
Scraping page: 13/13


In [205]:
df_rot.shape

(220, 8)

In [208]:
df_rot.tail()

,Rent,Available from,Surface,Interior,Number of bedrooms,Location,Description,url
15,3.000,right away,95,furnished,2,schinkelstraat 3061mn Rotterdam,furnished and luxurious 3 room apartment in ro...,https://www.huurda.com/properties_for_rent_in_...
16,0,right away,70,upholstered,4,-zuid putsebocht 3073 hs Rotterdam-zuid,wij bieden u een 2 kamer appartement gelegen a...,https://www.huurda.com/properties_for_rent_in_...
17,1.250,right away,65,furnished,2,schiedamsedijk 3011en Rotterdam,in a prime location well maintained 3 room fur...,https://www.huurda.com/properties_for_rent_in_...
18,1.500,right away,85,furnished,2,wijnbrugstraat 3011xw Rotterdam,lovely apartment located at wijnbrugstraat in...,https://www.huurda.com/properties_for_rent_in_...
19,0,right away,20,upholstered,1,-centrum korte schoonderloostraat 3024 tp Rott...,wij bieden u een gerenoveerde kamer gelegen in...,https://www.huurda.com/properties_for_rent_in_...


In [207]:
df_rot.to_csv(cfg_rot.data_path + 'rot_data.csv', encoding='utf-8', index=False)

### Den Haag

In [209]:
df_haag = main_scraper(cfg_haag.url, city='Den Haag', nb_page=12)

Scraping page: 1/12
Scraping page: 2/12
Scraping page: 3/12
https://www.huurda.com/properties_for_rent_in_Den_Haag/Upstairs_apartment/LandrÃ©straat--391694/
Scraping page: 4/12
Scraping page: 5/12
Scraping page: 6/12
Scraping page: 7/12
Scraping page: 8/12
Scraping page: 9/12
Scraping page: 10/12
Scraping page: 11/12
Scraping page: 12/12


In [210]:
df_haag.shape

(219, 8)

In [211]:
df_haag.tail()

,Rent,Available from,Surface,Interior,Number of bedrooms,Location,Description,url
15,775,right away,80,upholstered,2,tesselsestraat 2583jk Den Haag,"nice 4 room apartment (top floor), at a walkin...",https://www.huurda.com/properties_for_rent_in_...
16,1.150,right away,75,furnished,2,paul gabrielstraat 2596vb Den Haag,charming fully furnished 2 bedroom apartment s...,https://www.huurda.com/properties_for_rent_in_...
17,1.250,right away,55,upholstered,2,noordeinde 2514gr Den Haag,prachtig volledig gemeubileerd appartement met...,https://www.huurda.com/properties_for_rent_in_...
18,1.650,right away,85,furnished,2,bezuidenhoutseweg 2594ac Den Haag,"zeer luxe 3-kamer woning van ca.:85 m2, gelege...",https://www.huurda.com/properties_for_rent_in_...
19,2.750,right away,150,upholstered,5,hooikade 2514bk Den Haag,zeer ruime volledig gestoffeerde 5-kamerwoning...,https://www.huurda.com/properties_for_rent_in_...


In [25]:
df_haag.to_csv(cfg_haag.data_path + 'haag_data.csv', encoding='utf-8', index=False)

### Groningen

In [122]:
df_gro = main_scraper(cfg_gro.url, city='Groningen', nb_page=6)

Scraping page: 1/6
Scraping page: 2/6
Scraping page: 3/6
Scraping page: 4/6
Scraping page: 5/6
Scraping page: 6/6


In [123]:
df_gro.shape

(120, 8)

In [212]:
df_gro.head()

,Rent,Available from,Surface,Interior,Number of bedrooms,Location,Description,url
0,690,01-03-2019,27,upholstered,2,herebinnensingel 9711ge Groningen,two rooms available in student residence for 1...,https://www.huurda.com/properties_for_rent_in_...
1,1.490,01-03-2019,100,furnished possible,3,van starkenborghstraat 9721 ee Groningen,nice upstairs-house with three bedrooms in sou...,https://www.huurda.com/properties_for_rent_in_...
2,975,01-03-2019,190,upholstered,5,"se renting in stitswerd, jacob tilbusscherweg ...",detached house with stunning views! jacob til...,https://www.huurda.com/properties_for_rent_in_...
3,850,15-04-2019,55,furnished,1,witte de withstraat 9726ec Groningen,furnished house near center temporarily for re...,https://www.huurda.com/properties_for_rent_in_...
4,1.155,01-03-2019,85,furnished,2,petrus campersingel 9713ac Groningen,furnished rental with garden in groningen for ...,https://www.huurda.com/properties_for_rent_in_...


In [241]:
df_gro.to_csv(cfg_gro.data_path + 'gro_data.csv', encoding='utf-8', index=False)

### Utrecht

In [193]:
df_utr = main_scraper(cfg_utr.url, city='Utrecht', nb_page=10)

Scraping page: 1/10
Scraping page: 2/10
Scraping page: 3/10
Scraping page: 4/10
Scraping page: 5/10
Scraping page: 6/10
Scraping page: 7/10
Scraping page: 8/10
Scraping page: 9/10
Scraping page: 10/10


In [214]:
df_utr.shape

(200, 8)

In [216]:
df_utr.tail()

,Rent,Available from,Surface,Interior,Number of bedrooms,Location,Description,url
15,1.200,right away,55,furnished,1,lange nieuwstraat 3512pl Utrecht,this wonderful 2 room apartment is fully furni...,https://www.huurda.com/properties_for_rent_in_...
16,995,right away,130,upholstered,4,kasteeldrift 3436tr Utrecht,normal 021false false false nl x-none x-none ...,https://www.huurda.com/properties_for_rent_in_...
17,1.175,right away,60,furnished,1,amsterdamsestraatweg 3513ak Utrecht,:in 2007 renovated 2 bedroom apartment on the ...,https://www.huurda.com/properties_for_rent_in_...
18,1.250,right away,65,upholstered,1,janskerkhof 3512ad Utrecht,: fully furnished 2 room apartment. this apa...,https://www.huurda.com/properties_for_rent_in_...
19,1.650,right away,120,furnished,3,haverstraat 3511na Utrecht,,https://www.huurda.com/properties_for_rent_in_...


In [243]:
df_utr.to_csv(cfg_utr.data_path + 'utr_data.csv', encoding='utf-8', index=False)

### Eindhoven

In [219]:
df_ein = main_scraper(cfg_utr.url, city='Eindhoven', nb_page=9)

Scraping page: 1/9
Scraping page: 2/9
Scraping page: 3/9
Scraping page: 4/9
Scraping page: 5/9
Scraping page: 6/9
Scraping page: 7/9
Scraping page: 8/9
Scraping page: 9/9


In [220]:
df_ein.shape

(180, 8)

In [222]:
df_ein.tail()

,Rent,Available from,Surface,Interior,Number of bedrooms,Location,Description,url
15,800,right away,45,upholstered,1,"nt renting in utrecht, burgemeester van tuyllk...",aan een brede laan met boomrijke groenstrook l...,https://www.huurda.com/properties_for_rent_in_...
16,2.200,right away,152,furnished,3,"house renting in abcoude, bleekerhof rental ho...",an extremely bright and spacious alone-standin...,https://www.huurda.com/properties_for_rent_in_...
17,650,right away,60,furnished,2,"w renting in rhenen, nieuwe veenendaalseweg re...",,https://www.huurda.com/properties_for_rent_in_...
18,750,right away,30,upholstered,n.a.,"renting in utrecht, koppestokstraat rental hou...",nieuw op te leveren studio ca. 30m2. word gehe...,https://www.huurda.com/properties_for_rent_in_...
19,350,right away,20,upholstered,1,"house renting in nieuwegein, mr. j.m.m. hamers...","beschikbaar per direct, een zojuist nieuw gest...",https://www.huurda.com/properties_for_rent_in_...


In [244]:
df_ein.to_csv(cfg_ein.data_path + 'ein_data.csv', encoding='utf-8', index=False)

## Merge all data

In [229]:
df_list = [df_ams, df_rot, df_haag, df_gro, df_utr, df_ein]
df = pd.concat(df_list)

In [230]:
df.shape

(1094, 8)

In [231]:
df.head()

,Rent,Available from,Surface,Interior,Number of bedrooms,Location,Description,url
0,1.950,15-03-2019,86,furnished,2,knsm-laan 1019ld Amsterdam,modern and comfortable fully furnished apartme...,https://www.huurda.com/properties_for_rent_in_...
1,3.500,right away,130,furnished,4,cornelis anthoniszstraat 1071 vp Amsterdam,superb and completely renovated house in a pop...,https://www.huurda.com/properties_for_rent_in_...
2,1.995,right away,60,furnished,1,herengracht 1015 bt Amsterdam,fully furnished apartment in the middle of the...,https://www.huurda.com/properties_for_rent_in_...
4,1.675,11-03-2019,60,furnished,1,schinkelkade 1075 vh Amsterdam,on a top location in the old south near the vo...,https://www.huurda.com/properties_for_rent_in_...
5,1.450,right away,86,furnished,2,meeuwenlaan 1021 jc Amsterdam,the property is centrally located; pont ijplei...,https://www.huurda.com/properties_for_rent_in_...


## Save

In [232]:
df.to_csv(cfg_ams.data_path + 'crawled_data.csv', encoding='utf-8', index=False)

## Load

In [234]:
data = pd.read_csv(cfg_ams.data_path + 'crawled_data.csv', encoding='utf-8', engine='python')

In [235]:
data.shape

(1094, 8)

In [236]:
data

,Rent,Available from,Surface,Interior,Number of bedrooms,Location,Description,url
0,1.950,15-03-2019,86,furnished,2,knsm-laan 1019ld Amsterdam,modern and comfortable fully furnished apartme...,https://www.huurda.com/properties_for_rent_in_...
1,3.500,right away,130,furnished,4,cornelis anthoniszstraat 1071 vp Amsterdam,superb and completely renovated house in a pop...,https://www.huurda.com/properties_for_rent_in_...
2,1.995,right away,60,furnished,1,herengracht 1015 bt Amsterdam,fully furnished apartment in the middle of the...,https://www.huurda.com/properties_for_rent_in_...
3,1.675,11-03-2019,60,furnished,1,schinkelkade 1075 vh Amsterdam,on a top location in the old south near the vo...,https://www.huurda.com/properties_for_rent_in_...
4,1.450,right away,86,furnished,2,meeuwenlaan 1021 jc Amsterdam,the property is centrally located; pont ijplei...,https://www.huurda.com/properties_for_rent_in_...
5,1.700,right away,68,furnished,2,legmeerplein 1058 nl Amsterdam,this fantastic property is available immediate...,https://www.huurda.com/properties_for_rent_in_...
6,1.600,right away,80,furnished,2,loenermark 1025 ts Amsterdam,"apartment for rent, € 1.600 per month excl g/w...",https://www.huurda.com/properties_for_rent_in_...
7,2.950,01-03-2019,125,upholstered,3,archimedesweg 1098jr Amsterdam,beautiful and very bright house completely ren...,https://www.huurda.com/properties_for_rent_in_...
8,1.350,28-02-2019,32,furnished,1,tweede jacob van campenstraat 1073 xr Amsterdam,the house is in the heart of the popular distr...,https://www.huurda.com/properties_for_rent_in_...
9,1.850,right away,73,upholstered,2,hectorstraat 1076 pn Amsterdam,conferencing and completely renovated 3-room a...,https://www.huurda.com/properties_for_rent_in_...
